## ⚠️ BEFORE YOU START

**What day is it today?**

- ✅ **Friday after 4pm ET:** Perfect! Run all cells below
- ❌ **Monday-Thursday:** STOP! Wait until Friday after market close
- ❌ **Friday before 4pm:** Wait until market closes at 4pm ET

**Why Friday close?** GHB Strategy uses weekly closing prices. Trading mid-week gives false signals and won't match backtested performance.

---

# GHB Strategy Portfolio Scanner
**Gold-Gray-Blue Weekly Trading System**

## 📅 WEEKLY TRADING SCHEDULE

⚠️ **IMPORTANT:** Only run this scanner on **FRIDAY after 4pm ET** (after market close)

### Your Weekly Routine:

**FRIDAY (After 4pm ET)**
- 📊 Run this notebook (all cells)
- 📋 Review signals: BUY (P1), HOLD (P2/N1), SELL (N2)
- 📝 Make your trade list for Monday
- ⏱️ Time: 10-15 minutes

**WEEKEND (Saturday/Sunday)**
- 💭 Review and confirm your plan
- 🧮 Calculate position sizes (8-10% each)
- ✅ Prepare for Monday execution

**MONDAY (Market Open - 9:30am ET)**
- 🔵 **FIRST:** Execute ALL sell signals (N2 stocks)
- 🟡 **THEN:** Enter new buy positions (P1 stocks - top 3-5)
- ⏱️ Time: 15-30 minutes

---

**Last Run:** {current_date}  
**Strategy:** GHB Strategy (Gold-Gray-Blue)  
**Universe:** 25 Optimized Stocks (Your Watchlist + Top Performers)  
**Expected Annual Return:** +514%

In [1]:
# Import Required Libraries
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries loaded successfully")
print(f"📅 Current Date: {datetime.now().strftime('%Y-%m-%d')}")

✅ Libraries loaded successfully
📅 Current Date: 2026-01-15


## 1. Define Stock Universe

The 25 stocks selected for optimal performance:
- Your 12-stock watchlist (core holdings)
- 13 top performers from backtesting (2021-2025)
- Expected: +514% annual returns, ~14 trades/year

This delivers 117% of full universe returns with much easier management!

In [2]:
# GHB Strategy Optimized Portfolio - 25 Stocks
# Core: Your 12-stock watchlist + Top 13 performers from backtesting
GHB_UNIVERSE = [
    'ALAB', 'AMAT', 'AMD', 'ARM', 'ASML', 'AVGO', 'BKNG', 'CEG', 
    'COST', 'DASH', 'FANG', 'FTNT', 'GOOG', 'GOOGL', 'META', 'MRNA',
    'MRVL', 'MSFT', 'MU', 'NFLX', 'NVDA', 'PANW', 'PLTR', 'TSLA', 'TSM'
]

print(f"📊 Universe: {len(GHB_UNIVERSE)} stocks (Optimized Portfolio)")
print(f"📈 Stocks: {', '.join(sorted(GHB_UNIVERSE[:10]))}...")
print(f"💡 Expected: +514% annual return, ~14 trades/year")

📊 Universe: 25 stocks (Optimized Portfolio)
📈 Stocks: ALAB, AMAT, AMD, ARM, ASML, AVGO, BKNG, CEG, COST, DASH...
💡 Expected: +514% annual return, ~14 trades/year


## 2. Calculate Weekly Larsson States

For each stock, calculate:
- **Weekly Close** (Friday)
- **200-Day SMA** (D200)
- **4-Week ROC** (Rate of Change)
- **Weekly State** (P1/P2/N1/N2)

In [3]:
def calculate_weekly_larsson_state(ticker):
    """
    Calculate weekly Larsson state for a ticker
    Returns: dict with ticker data or None if error
    """
    try:
        # Download 1 year of daily data
        stock = yf.Ticker(ticker)
        df = stock.history(period='1y', interval='1d')
        
        if df.empty or len(df) < 200:
            return None
        
        # Get latest close
        close = df['Close'].iloc[-1]
        
        # Calculate 200-day SMA
        d200 = df['Close'].rolling(window=200).mean().iloc[-1]
        
        # Calculate 4-week ROC (20 trading days)
        if len(df) >= 20:
            price_4w_ago = df['Close'].iloc[-20]
            roc_4w = ((close - price_4w_ago) / price_4w_ago) * 100
        else:
            roc_4w = 0
        
        # Calculate distance from D200
        distance_pct = ((close - d200) / d200) * 100
        
        # Determine Weekly Larsson state (Strategy D rules)
        if close > d200:
            # Price above D200
            if roc_4w > 5 or distance_pct > 10:
                state = 'P1'  # Strong bullish
                signal = '🟡 BUY'
            else:
                state = 'P2'  # Consolidation
                signal = '⚪ HOLD'
        else:
            # Price below D200
            if distance_pct > -5:
                state = 'N1'  # Shallow pullback
                signal = '⚪ HOLD'
            else:
                state = 'N2'  # Downtrend
                signal = '🔵 SELL'
        
        return {
            'Ticker': ticker,
            'Close': close,
            'D200': d200,
            'Distance_%': distance_pct,
            'ROC_4W_%': roc_4w,
            'State': state,
            'Signal': signal
        }
        
    except Exception as e:
        print(f"❌ Error processing {ticker}: {str(e)}")
        return None

print("✅ Calculation function defined")

✅ Calculation function defined


## 3. Scan All 39 Stocks

This will take 1-2 minutes to download data and calculate states...

In [4]:
print(f"🔄 Scanning {len(GHB_UNIVERSE)} stocks... Please wait...\n")

results = []
for i, ticker in enumerate(GHB_UNIVERSE, 1):
    print(f"  [{i:2d}/{len(GHB_UNIVERSE)}] Processing {ticker:6s}...", end='\r')
    result = calculate_weekly_larsson_state(ticker)
    if result:
        results.append(result)

df_results = pd.DataFrame(results)

print(f"\n✅ Scan complete! Processed {len(df_results)}/{len(GHB_UNIVERSE)} stocks")
print(f"❌ Failed: {len(GHB_UNIVERSE) - len(df_results)} stocks")

🔄 Scanning 25 stocks... Please wait...

  [25/25] Processing TSM   ...
✅ Scan complete! Processed 25/25 stocks
❌ Failed: 0 stocks


## 4. Strategy D Signals

### Buy Signals (P1 - Gold)
**Action:** Enter new positions or add to existing  
**Requirements:** Price > D200 + Strong momentum

In [5]:
# Filter P1 (BUY) signals
p1_signals = df_results[df_results['State'] == 'P1'].sort_values('ROC_4W_%', ascending=False)

print("=" * 100)
print(f"🟡 P1 (GOLD) - BUY SIGNALS: {len(p1_signals)} stocks")
print("=" * 100)

if len(p1_signals) > 0:
    print(f"\n{'Ticker':<8} {'Price':<10} {'D200':<10} {'Above D200':<12} {'4W ROC':<10} {'Strength':<15}")
    print("-" * 100)
    
    for _, row in p1_signals.iterrows():
        # Determine strength
        if row['Distance_%'] > 30:
            strength = "🔥 EXPLOSIVE"
        elif row['Distance_%'] > 20:
            strength = "💪 VERY STRONG"
        elif row['ROC_4W_%'] < 0:
            strength = "✅ PULLBACK BUY"
        else:
            strength = "✅ STRONG"
        
        print(f"{row['Ticker']:<8} ${row['Close']:<9.2f} ${row['D200']:<9.2f} {row['Distance_%']:>+10.1f}% {row['ROC_4W_%']:>+8.2f}% {strength:<15}")
    
    print("\n💡 RECOMMENDATION:")
    print("   - Enter NEW positions in these P1 stocks")
    print("   - Add to existing positions if capital available")
    print("   - Prioritize: Explosive > Very Strong > Strong > Pullback Buy")
else:
    print("\n⚠️  No P1 buy signals this week")
    print("   - Wait for new P1 entries")
    print("   - Hold existing positions")

🟡 P1 (GOLD) - BUY SIGNALS: 14 stocks

Ticker   Price      D200       Above D200   4W ROC     Strength       
----------------------------------------------------------------------------------------------------
MU       $342.20    $159.90        +114.0%   +51.80% 🔥 EXPLOSIVE    
ASML     $1336.35   $868.29         +53.9%   +31.60% 🔥 EXPLOSIVE    
AMAT     $323.43    $198.11         +63.3%   +30.27% 🔥 EXPLOSIVE    
MRNA     $39.60     $27.57          +43.7%   +29.79% 🔥 EXPLOSIVE    
TSM      $346.57    $244.80         +41.6%   +25.13% 🔥 EXPLOSIVE    
ALAB     $172.12    $139.10         +23.7%   +22.73% 💪 VERY STRONG  
AMD      $235.54    $168.08         +40.1%   +18.89% 🔥 EXPLOSIVE    
GOOGL    $332.27    $225.24         +47.5%   +11.98% 🔥 EXPLOSIVE    
GOOG     $332.76    $226.22         +47.1%   +11.64% 🔥 EXPLOSIVE    
COST     $956.82    $948.78          +0.8%   +10.92% ✅ STRONG       
NVDA     $186.95    $163.66         +14.2%    +9.37% ✅ STRONG       
AVGO     $346.71    $296.26    

### Hold Signals (P2 & N1 - Gray)
**Action:** Continue holding existing positions  
**Meaning:** Normal consolidation/pullback in trend

In [6]:
# Filter P2 and N1 (HOLD) signals
hold_signals = df_results[df_results['State'].isin(['P2', 'N1'])].sort_values('Distance_%', ascending=False)

print("=" * 100)
print(f"⚪ P2/N1 (GRAY) - HOLD SIGNALS: {len(hold_signals)} stocks")
print("=" * 100)

if len(hold_signals) > 0:
    print(f"\n{'Ticker':<8} {'Price':<10} {'D200':<10} {'Distance':<12} {'4W ROC':<10} {'State':<8} {'Status':<20}")
    print("-" * 100)
    
    for _, row in hold_signals.iterrows():
        if row['State'] == 'P2':
            status = "Consolidation"
        else:
            status = "Shallow Pullback"
        
        print(f"{row['Ticker']:<8} ${row['Close']:<9.2f} ${row['D200']:<9.2f} {row['Distance_%']:>+10.1f}% {row['ROC_4W_%']:>+8.2f}% {row['State']:<8} {status:<20}")
    
    print("\n💡 RECOMMENDATION:")
    print("   - HOLD all existing positions")
    print("   - Do NOT sell - this is normal consolidation")
    print("   - Watch for transition to P1 (upgrade) or N2 (downgrade)")
else:
    print("\n✅ No stocks in consolidation phase")

⚪ P2/N1 (GRAY) - HOLD SIGNALS: 6 stocks

Ticker   Price      D200       Distance     4W ROC     State    Status              
----------------------------------------------------------------------------------------------------
MRVL     $82.08     $75.05           +9.4%    +0.54% P2       Consolidation       
CEG      $345.33    $318.23          +8.5%    +1.28% P2       Consolidation       
FANG     $151.84    $141.76          +7.1%    -1.45% P2       Consolidation       
PANW     $192.67    $192.80          -0.1%    +5.03% N1       Shallow Pullback    
BKNG     $5181.67   $5271.25         -1.7%    -2.98% N1       Shallow Pullback    
MSFT     $458.05    $480.66          -4.7%    -3.79% N1       Shallow Pullback    

💡 RECOMMENDATION:
   - HOLD all existing positions
   - Do NOT sell - this is normal consolidation
   - Watch for transition to P1 (upgrade) or N2 (downgrade)


### Sell Signals (N2 - Blue)
**Action:** Exit positions immediately  
**Requirements:** Price < D200 + Weak momentum  
**⚠️ CRITICAL: Execute these sells on Monday!**

In [7]:
# Filter N2 (SELL) signals
n2_signals = df_results[df_results['State'] == 'N2'].sort_values('Distance_%', ascending=True)

print("=" * 100)
print(f"🔵 N2 (BLUE) - SELL SIGNALS: {len(n2_signals)} stocks")
print("=" * 100)

if len(n2_signals) > 0:
    print(f"\n⚠️  EXIT THESE POSITIONS ON MONDAY!\n")
    print(f"{'Ticker':<8} {'Price':<10} {'D200':<10} {'Below D200':<12} {'4W ROC':<10} {'Severity':<15}")
    print("-" * 100)
    
    for _, row in n2_signals.iterrows():
        # Determine severity
        if row['Distance_%'] < -20:
            severity = "🚨 SEVERE"
        elif row['Distance_%'] < -10:
            severity = "⚠️  MAJOR"
        else:
            severity = "📉 MINOR"
        
        print(f"{row['Ticker']:<8} ${row['Close']:<9.2f} ${row['D200']:<9.2f} {row['Distance_%']:>+10.1f}% {row['ROC_4W_%']:>+8.2f}% {severity:<15}")
    
    print("\n💡 ACTION REQUIRED:")
    print("   - SELL all N2 positions on Monday at market open")
    print("   - Do NOT wait for bounce - trend is broken")
    print("   - Preserve capital for new P1 opportunities")
else:
    print("\n✅ No sell signals - all positions healthy!")

🔵 N2 (BLUE) - SELL SIGNALS: 5 stocks

⚠️  EXIT THESE POSITIONS ON MONDAY!

Ticker   Price      D200       Below D200   4W ROC     Severity       
----------------------------------------------------------------------------------------------------
ARM      $107.06    $137.20         -22.0%    -6.57% 🚨 SEVERE       
NFLX     $89.12     $113.11         -21.2%    -5.98% 🚨 SEVERE       
FTNT     $77.59     $90.28          -14.1%    -2.25% ⚠️  MAJOR      
META     $618.33    $674.65          -8.3%    -4.80% 📉 MINOR        
DASH     $210.16    $228.51          -8.0%    -5.03% 📉 MINOR        

💡 ACTION REQUIRED:
   - SELL all N2 positions on Monday at market open
   - Do NOT wait for bounce - trend is broken
   - Preserve capital for new P1 opportunities


## 5. Weekly Summary

Quick overview of portfolio status and action items

In [8]:
print("=" * 100)
print("📊 GHB STRATEGY WEEKLY SUMMARY")
print("=" * 100)

print(f"\n🟡 BUY Signals (P1):  {len(p1_signals)} stocks")
print(f"⚪ HOLD Signals (P2/N1): {len(hold_signals)} stocks")
print(f"🔵 SELL Signals (N2): {len(n2_signals)} stocks")
print(f"📊 Total Scanned: {len(df_results)}/{len(GHB_UNIVERSE)} stocks")

# Calculate portfolio health
total_bullish = len(p1_signals)
total_neutral = len(hold_signals)
total_bearish = len(n2_signals)
total = len(df_results)

pct_bullish = (total_bullish / total * 100) if total > 0 else 0
pct_bearish = (total_bearish / total * 100) if total > 0 else 0

print(f"\n📈 Market Health:")
print(f"   Bullish: {pct_bullish:.1f}% ({total_bullish} stocks)")
print(f"   Neutral: {(total_neutral/total*100):.1f}% ({total_neutral} stocks)")
print(f"   Bearish: {pct_bearish:.1f}% ({total_bearish} stocks)")

if pct_bullish > 60:
    market_sentiment = "🟢 VERY BULLISH - Many opportunities"
elif pct_bullish > 40:
    market_sentiment = "🟡 BULLISH - Good opportunities"
elif pct_bullish > 20:
    market_sentiment = "🟠 NEUTRAL - Selective opportunities"
else:
    market_sentiment = "🔴 BEARISH - Few opportunities, preserve cash"

print(f"\n📊 Market Sentiment: {market_sentiment}")

# Action items
print("\n✅ ACTION ITEMS FOR THIS WEEK:")
if len(n2_signals) > 0:
    print(f"   1. MONDAY: Sell {len(n2_signals)} N2 positions at market open")
else:
    print("   1. No sells required")

if len(p1_signals) > 0:
    print(f"   2. MONDAY: Enter up to {min(5, len(p1_signals))} new P1 positions")
    print(f"      → Priority: {', '.join(p1_signals.head(5)['Ticker'].tolist())}")
else:
    print("   2. No new buys available - hold cash")

if len(hold_signals) > 0:
    print(f"   3. Monitor {len(hold_signals)} holding positions for state changes")
else:
    print("   3. No positions to monitor")

print("\n" + "=" * 100)

📊 GHB STRATEGY WEEKLY SUMMARY

🟡 BUY Signals (P1):  14 stocks
⚪ HOLD Signals (P2/N1): 6 stocks
🔵 SELL Signals (N2): 5 stocks
📊 Total Scanned: 25/25 stocks

📈 Market Health:
   Bullish: 56.0% (14 stocks)
   Neutral: 24.0% (6 stocks)
   Bearish: 20.0% (5 stocks)

📊 Market Sentiment: 🟡 BULLISH - Good opportunities

✅ ACTION ITEMS FOR THIS WEEK:
   1. MONDAY: Sell 5 N2 positions at market open
   2. MONDAY: Enter up to 5 new P1 positions
      → Priority: MU, ASML, AMAT, MRNA, TSM
   3. Monitor 6 holding positions for state changes



## 6. Detailed Stock Data

Full dataset for analysis and record-keeping

In [9]:
# Display full results sorted by state then distance
df_display = df_results.copy()
df_display['State_Order'] = df_display['State'].map({'P1': 1, 'P2': 2, 'N1': 3, 'N2': 4})
df_display = df_display.sort_values(['State_Order', 'Distance_%'], ascending=[True, False])
df_display = df_display.drop('State_Order', axis=1)

print("\n📋 COMPLETE SCAN RESULTS")
print("=" * 100)
print(df_display.to_string(index=False))
print("=" * 100)


📋 COMPLETE SCAN RESULTS
Ticker       Close        D200  Distance_%  ROC_4W_% State Signal
    MU  342.200012  159.904893  114.002214 51.799504    P1  🟡 BUY
  AMAT  323.427612  198.105358   63.260406 30.272529    P1  🟡 BUY
  ASML 1336.354980  868.291302   53.906296 31.604836    P1  🟡 BUY
 GOOGL  332.274994  225.244576   47.517423 11.982675    P1  🟡 BUY
  GOOG  332.760010  226.222133   47.094365 11.641955    P1  🟡 BUY
  MRNA   39.599998   27.565750   43.656525 29.793504    P1  🟡 BUY
   TSM  346.570587  244.803537   41.570907 25.133809    P1  🟡 BUY
   AMD  235.539993  168.083800   40.132478 18.893540    P1  🟡 BUY
  ALAB  172.119995  139.096300   23.741606 22.732450    P1  🟡 BUY
  TSLA  442.700012  368.299350   20.201138 -5.256174    P1  🟡 BUY
  AVGO  346.709991  296.257583   17.029913  6.549722    P1  🟡 BUY
  PLTR  178.259995  155.013425   14.996488  0.547127    P1  🟡 BUY
  NVDA  186.953506  163.661159   14.232056  9.367909    P1  🟡 BUY
  COST  956.816284  948.776876    0.847344 10.91592

## 7. Export Results

Save results to CSV for record-keeping and further analysis

In [10]:
# Archive old scan results
import os
import shutil
from pathlib import Path
from reportlab.lib import colors
from reportlab.lib.pagesizes import letter, A4
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer, PageBreak
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from reportlab.lib.enums import TA_CENTER, TA_LEFT

# Create folders if they don't exist
results_dir = Path("../ghb_scanner_results")
archive_dir = results_dir / "archive"
results_dir.mkdir(exist_ok=True)
archive_dir.mkdir(exist_ok=True)

# Move old CSV and PDF files to archive
for old_file in list(results_dir.glob("ghb_strategy_signals_*.csv")) + list(results_dir.glob("ghb_strategy_signals_*.pdf")):
    try:
        shutil.move(str(old_file), str(archive_dir / old_file.name))
        print(f"📦 Archived: {old_file.name}")
    except Exception as e:
        print(f"⚠️ Could not archive {old_file.name}: {e}")

# Save CSV with date and time
datetime_str = datetime.now().strftime('%Y%m%d_%H%M')
output_file = results_dir / f"ghb_strategy_signals_{datetime_str}.csv"
df_results.to_csv(output_file, index=False)

# Generate PDF Report with date and time
pdf_file = results_dir / f"ghb_strategy_signals_{datetime_str}.pdf"
doc = SimpleDocTemplate(str(pdf_file), pagesize=letter, topMargin=0.5*inch, bottomMargin=0.5*inch)
styles = getSampleStyleSheet()
story = []

# Title
title_style = ParagraphStyle('CustomTitle', parent=styles['Heading1'], fontSize=18, textColor=colors.HexColor('#1f4788'), alignment=TA_CENTER, spaceAfter=12)
story.append(Paragraph("GHB Strategy Portfolio Scanner", title_style))
subtitle_style = ParagraphStyle('Subtitle', parent=styles['Normal'], fontSize=10, textColor=colors.HexColor('#666666'), alignment=TA_CENTER, spaceAfter=6)
story.append(Paragraph("Gold-Gray-Blue Weekly Trading System", subtitle_style))
story.append(Paragraph(f"Scan Date: {datetime.now().strftime('%B %d, %Y at %I:%M %p')}", styles['Normal']))
story.append(Spacer(1, 0.2*inch))

# State Abbreviations Key
abbrev_style = ParagraphStyle('AbbrevTitle', parent=styles['Heading3'], fontSize=11, textColor=colors.HexColor('#2c5aa0'), spaceAfter=6)
story.append(Paragraph("State Abbreviations", abbrev_style))
abbrev_data = [
    ['State', 'Meaning', 'Signal'],
    ['P1 (Gold)', 'Positive + Strong: Price > 200-day SMA + momentum', 'BUY'],
    ['P2 (Gray)', 'Positive + Weak: Price > 200-day SMA, low momentum', 'HOLD'],
    ['N1 (Gray)', 'Negative + Strong: Price slightly below 200-day SMA', 'HOLD'],
    ['N2 (Blue)', 'Negative + Weak: Price well below 200-day SMA', 'SELL']
]
abbrev_table = Table(abbrev_data, colWidths=[1*inch, 3.5*inch, 0.8*inch])
abbrev_table.setStyle(TableStyle([
    ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#e8f0f8')),
    ('TEXTCOLOR', (0, 0), (-1, 0), colors.HexColor('#1f4788')),
    ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
    ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
    ('FONTSIZE', (0, 0), (-1, 0), 9),
    ('BOTTOMPADDING', (0, 0), (-1, 0), 6),
    ('GRID', (0, 0), (-1, -1), 0.5, colors.grey),
    ('FONTSIZE', (0, 1), (-1, -1), 8),
    ('TOPPADDING', (0, 1), (-1, -1), 4),
    ('BOTTOMPADDING', (0, 1), (-1, -1), 4),
    ('BACKGROUND', (0, 1), (0, 1), colors.HexColor('#fff9cc')),  # P1 - Gold
    ('BACKGROUND', (0, 2), (0, 2), colors.lightgrey),  # P2 - Gray
    ('BACKGROUND', (0, 3), (0, 3), colors.lightgrey),  # N1 - Gray
    ('BACKGROUND', (0, 4), (0, 4), colors.HexColor('#cce5ff')),  # N2 - Blue
]))
story.append(abbrev_table)
story.append(Spacer(1, 0.2*inch))

# Summary Section
summary_style = ParagraphStyle('Summary', parent=styles['Heading2'], fontSize=14, textColor=colors.HexColor('#2c5aa0'))
story.append(Paragraph("Weekly Summary", summary_style))
story.append(Spacer(1, 0.1*inch))

total_bullish = len(p1_signals)
total_neutral = len(hold_signals)
total_bearish = len(n2_signals)
total = len(df_results)
pct_bullish = (total_bullish / total * 100) if total > 0 else 0

summary_data = [
    ['Signal Type', 'Count', 'Percentage'],
    [f'BUY (P1 - Gold)', str(total_bullish), f'{pct_bullish:.1f}%'],
    [f'HOLD (P2/N1 - Gray)', str(total_neutral), f'{(total_neutral/total*100):.1f}%'],
    [f'SELL (N2 - Blue)', str(total_bearish), f'{(total_bearish/total*100):.1f}%'],
    ['Total Scanned', str(total), '100%']
]

summary_table = Table(summary_data, colWidths=[3*inch, 1*inch, 1.5*inch])
summary_table.setStyle(TableStyle([
    ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#1f4788')),
    ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
    ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
    ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
    ('FONTSIZE', (0, 0), (-1, 0), 11),
    ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
    ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
    ('GRID', (0, 0), (-1, -1), 1, colors.black),
    ('FONTSIZE', (0, 1), (-1, -1), 10),
    ('TOPPADDING', (0, 1), (-1, -1), 6),
    ('BOTTOMPADDING', (0, 1), (-1, -1), 6),
]))
story.append(summary_table)
story.append(Spacer(1, 0.2*inch))

# Market Sentiment
if pct_bullish > 60:
    sentiment = "VERY BULLISH - Many opportunities"
elif pct_bullish > 40:
    sentiment = "BULLISH - Good opportunities"
elif pct_bullish > 20:
    sentiment = "NEUTRAL - Selective opportunities"
else:
    sentiment = "BEARISH - Few opportunities, preserve cash"

story.append(Paragraph(f"<b>Market Sentiment:</b> {sentiment}", styles['Normal']))
story.append(Spacer(1, 0.2*inch))

# Action Items
story.append(Paragraph("Action Items for Monday", summary_style))
story.append(Spacer(1, 0.1*inch))

action_items = []
if len(n2_signals) > 0:
    action_items.append(f"1. SELL {len(n2_signals)} N2 positions at market open: {', '.join(n2_signals['Ticker'].tolist())}")
else:
    action_items.append("1. No sells required")

if len(p1_signals) > 0:
    top_buys = ', '.join(p1_signals.head(5)['Ticker'].tolist())
    action_items.append(f"2. Enter up to {min(5, len(p1_signals))} new P1 positions. Priority: {top_buys}")
else:
    action_items.append("2. No new buys available - hold cash")

if len(hold_signals) > 0:
    action_items.append(f"3. Monitor {len(hold_signals)} holding positions for state changes")

for item in action_items:
    story.append(Paragraph(item, styles['Normal']))
    story.append(Spacer(1, 0.05*inch))

story.append(PageBreak())

# Detailed Results by Category
story.append(Paragraph("Detailed Stock Analysis", summary_style))
story.append(Spacer(1, 0.1*inch))

# Function to create table for each category
def create_category_table(df_category, title, bg_color):
    if len(df_category) == 0:
        return None
    
    # Sort by ROC for P1, by Distance for others
    if title.startswith('BUY'):
        df_category = df_category.sort_values('ROC_4W_%', ascending=False)
    else:
        df_category = df_category.sort_values('Distance_%', ascending=False)
    
    table_data = [['Ticker', 'Price', 'D200', 'Dist %', 'ROC %', 'State']]
    for _, row in df_category.iterrows():
        table_data.append([
            row['Ticker'],
            f"${row['Close']:.2f}",
            f"${row['D200']:.2f}",
            f"{row['Distance_%']:+.1f}%",
            f"{row['ROC_4W_%']:+.1f}%",
            row['State']
        ])
    
    table = Table(table_data, colWidths=[0.8*inch, 1*inch, 1*inch, 0.9*inch, 0.9*inch, 0.7*inch])
    table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#1f4788')),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, 0), 9),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 8),
        ('BACKGROUND', (0, 1), (-1, -1), bg_color),
        ('GRID', (0, 0), (-1, -1), 0.5, colors.grey),
        ('FONTSIZE', (0, 1), (-1, -1), 8),
        ('TOPPADDING', (0, 1), (-1, -1), 4),
        ('BOTTOMPADDING', (0, 1), (-1, -1), 4),
    ]))
    return table

# 1. BUY Signals (P1)
story.append(Paragraph(f"1. BUY Signals - P1 (Gold) [{len(p1_signals)} stocks]", abbrev_style))
story.append(Paragraph("Action: Enter new positions or add to existing. Sorted by momentum (highest first).", styles['Normal']))
story.append(Spacer(1, 0.05*inch))
if len(p1_signals) > 0:
    buy_table = create_category_table(p1_signals, 'BUY', colors.HexColor('#fff9cc'))
    story.append(buy_table)
else:
    story.append(Paragraph("No BUY signals this week.", styles['Normal']))
story.append(Spacer(1, 0.15*inch))

# 2. HOLD Signals (P2/N1)
story.append(Paragraph(f"2. HOLD Signals - P2/N1 (Gray) [{len(hold_signals)} stocks]", abbrev_style))
story.append(Paragraph("Action: Continue holding. Monitor for state changes (upgrade to P1 or downgrade to N2).", styles['Normal']))
story.append(Spacer(1, 0.05*inch))
if len(hold_signals) > 0:
    hold_table = create_category_table(hold_signals, 'HOLD', colors.lightgrey)
    story.append(hold_table)
else:
    story.append(Paragraph("No HOLD positions.", styles['Normal']))
story.append(PageBreak())

# 3. SELL Signals (N2)
story.append(Paragraph(f"3. SELL Signals - N2 (Blue) [{len(n2_signals)} stocks]", abbrev_style))
story.append(Paragraph("Action: EXIT immediately at Monday market open. Trend is broken.", styles['Normal']))
story.append(Spacer(1, 0.05*inch))
if len(n2_signals) > 0:
    sell_table = create_category_table(n2_signals, 'SELL', colors.HexColor('#ffe5e5'))
    story.append(sell_table)
else:
    story.append(Paragraph("No SELL signals - all positions healthy!", styles['Normal']))

# Build PDF
doc.build(story)

print(f"\n✅ Results saved:")
print(f"   📊 CSV: {output_file.name}")
print(f"   📄 PDF: {pdf_file.name}")
print(f"📁 Location: {output_file.parent.absolute()}")
print(f"📂 Old scans archived to: {archive_dir.name}/")

📦 Archived: ghb_strategy_signals_20260115_1646.csv
📦 Archived: ghb_strategy_signals_20260115_1646.pdf

✅ Results saved:
   📊 CSV: ghb_strategy_signals_20260115_1648.csv
   📄 PDF: ghb_strategy_signals_20260115_1648.pdf
📁 Location: c:\workspace\portfolio_analyser\notebooks\..\ghb_scanner_results
📂 Old scans archived to: archive/


## GHB Strategy Quick Reference

### Your Optimized Portfolio
**25 stocks:** Your 12 watchlist + 13 top performers  
**Expected:** +514% annual return, ~14 trades/year, 57% win rate

### Entry Rules (BUY)
- State = P1 (Gold)
- Price > 200-day SMA
- Strong momentum (ROC > 5% OR distance > 10%)

### Hold Rules
- **P1 (Gold):** Continue holding, consider adding
- **P2 (Gray):** Hold through consolidation
- **N1 (Gray):** Hold through shallow pullback

### Exit Rules (SELL)
- State = N2 (Blue)
- Price < 200-day SMA
- Weak momentum
- **Execute Monday at open!**

### Expected Performance
- **Annual Return:** +514% (optimized portfolio)
- **Trades Per Year:** ~14 (1-2 per month)
- **Win Rate:** 57%
- **Avg Win:** +64%
- **Avg Loss:** -11%
- **Hold Period:** 8-12 months

### Risk Management
- Max 8-10% per position
- 5-7 concurrent positions typical
- 20-30% cash reserve
- Weekly monitoring only (10-15 minutes)

---
**Next Steps:**
1. Review signals above
2. Execute trades Monday at open
3. Run this notebook again next Friday
4. Track results in portfolio tracker

**Documentation:** See `docs/GHB_STRATEGY_GUIDE.md` for complete strategy details  
**Portfolio List:** See `data/ghb_optimized_portfolio.txt` for your 25 stocks